<a href="https://colab.research.google.com/github/kattens/BioPDBKit/blob/main/Downloading_the_pdb_files_from_the_lists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython
!pip install freesasa biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for freesasa: filename=freesasa-2.2.1-cp310-cp310-linux_x86_64.whl size=853073 sha256=a7e99e0dd9eb805ce4cd7307a05e020b1b8f68623a97c414d57ca885a7743c22
  Stored in directory: /root/.cache/pip/wheels/ef/f5/89/77b77735ac804410dd420a5a8ed504321ad051cf65dd3d1f9f
Successfully built freesasa


In [ ]:
import os
import requests
import csv
from Bio import PDB  # Import the Bio.PDB module
import freesasa

In [ ]:
input_file = "/content/drive/MyDrive/pdb_entry_files.csv"

first_column_values = []

with open(input_file, 'r') as csv_in:
    reader = csv.reader(csv_in)
    header = next(reader)  # Skip the header

    for row in reader:
        first_column_values.append(row[0])  # Append the value from the first column

print(first_column_values)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/pdb_entry_files.csv'

In [ ]:
# Define the PDBDownloader class with an output directory parameter
class PDBDownloader:
    def __init__(self, output_directory):
        self.output_directory = output_directory

    def download_pdb(self, protein_name_list):
        for protein_name in protein_name_list:
            filename = f"{protein_name}.pdb"
            file_path = os.path.join(self.output_directory, filename)

            # Check if the file already exists in the output directory
            if os.path.exists(file_path):
                print(f"File already exists for {protein_name}, skipping download.")
                continue

            url = f"https://files.rcsb.org/download/{protein_name}.pdb"

            try:
                response = requests.get(url)
                response.raise_for_status()
                pdb_content = response.text

                # Check if the response contains the PDB content
                if "HEADER    " not in pdb_content:
                    print(f"No PDB file found for {protein_name}")
                    continue

                with open(file_path, "w") as file:
                    file.write(pdb_content)

                print(f"Downloaded PDB file for {protein_name} to {self.output_directory}")
            except requests.HTTPError as e:
                print(f"Failed to download PDB file for {protein_name}")
                print(f"HTTP Error: {e}")
            except Exception as e:
                print(f"Failed to download PDB file for {protein_name}")
                print(f"Error: {e}")


In [ ]:
# Specify the output directory
output_directory = "/content/drive/MyDrive/NewLatestPDBFiles"

# Create an instance of PDBDownloader
downloader = PDBDownloader(output_directory)
# Download the PDB files
downloader.download_pdb(first_column_values)


#Start from here

In [ ]:
import glob
path = '/content/drive/MyDrive/PDBFILES/PDBs'

#files = glob.glob(os.path.join(output_directory, '*'))

files = glob.glob(os.path.join(path, '*'))
total_files = len([f for f in files if os.path.isfile(f)])

In [ ]:
total_files

6

In [ ]:
import freesasa
import os
import glob

# Directory where your PDB files are located
pdb_directory = '/content/drive/MyDrive/PDBFILES/PDBs'

# Function to calculate SASA for each residue
def calculate_sasa_per_residue(pdb_file):
    structure = freesasa.Structure(pdb_file)
    result = freesasa.calc(structure)

    sasa_per_residue = []
    for i in range(structure.nAtoms()):
        atom = structure.atomName(i)
        resname = structure.residueName(i)
        resnum = structure.residueNumber(i)
        chain = structure.chainLabel(i)
        sasa = result.atomArea(i)
        residue_key = f"{chain}{resnum} {resname}"

        # Find if this residue_key already exists in sasa_per_residue
        found = False
        for residue in sasa_per_residue:
            if residue['residue_key'] == residue_key:
                residue['sasa'] += sasa
                found = True
                break

        if not found:
            sasa_per_residue.append({'residue_key': residue_key, 'sasa': sasa})

    return sasa_per_residue

# Find all .pdb files in the directory
pdb_files = glob.glob(os.path.join(pdb_directory, '*.pdb'))

# Store SASA values for each file in a dictionary
sasa_results = {}

# Loop through each PDB file and calculate SASA per residue
for pdb_file in pdb_files:
    file_name = os.path.basename(pdb_file)
    sasa_per_residue = calculate_sasa_per_residue(pdb_file)
    sasa_results[file_name] = sasa_per_residue

# sasa_results now contains the SASA values for each residue in each PDB file


In [ ]:
sasa_results['1H0J.pdb']

[{'residue_key': 'A   1  LEU', 'sasa': 65.16777016823494},
 {'residue_key': 'A   2  LYS', 'sasa': 86.37586859683762},
 {'residue_key': 'A   3  CYS', 'sasa': 0.2736108388925816},
 {'residue_key': 'A   4  ASN', 'sasa': 10.342897289822774},
 {'residue_key': 'A   5  LYS', 'sasa': 89.63320079161954},
 {'residue_key': 'A   6  LEU', 'sasa': 56.69872835603198},
 {'residue_key': 'A   7  VAL', 'sasa': 79.09488883346147},
 {'residue_key': 'A   8  PRO', 'sasa': 59.51950687566779},
 {'residue_key': 'A   9  LEU', 'sasa': 134.62982242537984},
 {'residue_key': 'A  10  PHE', 'sasa': 109.65115309092035},
 {'residue_key': 'A  11  TYR', 'sasa': 99.25195677867046},
 {'residue_key': 'A  12  LYS', 'sasa': 94.38977162837082},
 {'residue_key': 'A  13  THR', 'sasa': 95.75311504234452},
 {'residue_key': 'A  14  CYS', 'sasa': 17.81685829434199},
 {'residue_key': 'A  15  PRO', 'sasa': 73.03760500216183},
 {'residue_key': 'A  16  ALA', 'sasa': 111.12249371819176},
 {'residue_key': 'A  17  GLY', 'sasa': 64.147051902

since this part of the codeis seperated from the downloader part, we just copy and paste the code then call it here.

In [ ]:
import multiprocessing
import re
from Bio.PDB.Polypeptide import is_aa
from Bio.PDB import PDBParser, PPBuilder, DSSP
from Bio.PDB.DSSP import DSSP
from collections import defaultdict
import freesasa

class PDBProcessor:
    def __init__(self, file_path):
        self.file_path = file_path  # Store the file path
        self.structure = PDBParser(QUIET=True).get_structure('PDB', file_path)
        with open(file_path, 'r') as f:
            self.pdb_lines = f.readlines()

    def protein_name(self):
        for model in self.structure:
            for chain in model:
                for residue in chain:
                    if is_aa(residue):
                        return residue.id[0]
        return "N/A"

    def polymer_entity(self):
        criteria = {"DNA": False, "Protein": False, "RNA": False}
        for model in self.structure:
            for chain in model:
                for residue in chain:
                    if residue.resname in ["DA", "DT", "DC", "DG"]:
                        criteria["DNA"] = True
                    elif residue.resname in ["A", "U", "C", "G"]:
                        criteria["RNA"] = True
                    elif is_aa(residue, standard=True):
                        criteria["Protein"] = True
        return ('DNA' if criteria["DNA"] else '') + \
               ('RNA' if criteria["RNA"] else '') + \
               ('Protein' if criteria["Protein"] else '') or 'Unknown'

    def sequence(self):
        sequences = {}
        ppb = PPBuilder()
        for pp in ppb.build_peptides(self.structure):
            sequences['Protein'] = str(pp.get_sequence())
        for model in self.structure:
            for chain in model:
                dna_sequence = []
                rna_sequence = []
                for residue in chain:
                    if residue.resname in ["DA", "DT", "DC", "DG"]:
                        dna_sequence.append(residue.resname)
                    elif residue.resname in ["A", "U", "C", "G"]:
                        rna_sequence.append(residue.resname)
                if dna_sequence:
                    sequences[f'{chain.id}_DNA'] = ''.join(dna_sequence)
                if rna_sequence:
                    sequences[f'{chain.id}_RNA'] = ''.join(rna_sequence)
        return sequences

    def c_alpha_coords(self):
        c_alpha_coords = {}
        for model in self.structure:
            for chain in model:
                chain_coords = []
                for residue in chain:
                    if "CA" in residue:
                        ca_atom = residue["CA"]
                        chain_coords.append(ca_atom.coord)
                if chain_coords:
                    c_alpha_coords[chain.id] = chain_coords
        return c_alpha_coords if c_alpha_coords else "Cα coordinates not found"

    def refinement_resolution(self):
        resolution = self.structure.header.get("resolution", "Resolution information not found")
        if resolution == "Resolution information not found":
            for line in self.pdb_lines:
                if line.startswith("REMARK   2 RESOLUTION."):
                    resolution = line.split()[-2]
                    break
        return resolution

    def experiment_type(self):
        exp_type = self.structure.header.get("structure_method", "Structure method information not found")
        if exp_type == "Structure method information not found":
            for line in self.pdb_lines:
                if line.startswith("EXPDTA"):
                    exp_type = " ".join(line.split()[1:])
                    break
        return exp_type

    def enzyme_classification(self):
        ec = self.structure.header.get("compound", {}).get("ec", "Enzyme Classification not found")
        if ec == "Enzyme Classification not found":
            # PDB files typically don't contain enzyme classification. You'd need to access an external database for this.
            ec = "N/A"
        return ec

    def symmetry_type(self):
        sym_type = self.structure.header.get("symmetry", "Symmetry Type not found")
        if sym_type == "Symmetry Type not found":
            for line in self.pdb_lines:
                if line.startswith("CRYST1"):
                    sym_type = line[55:].strip()
                    break
        return sym_type

    def r_factor(self):
        for line in self.pdb_lines:
            if "REMARK   3   R VALUE" in line:
                r_factor = line.split()[-1]
                return r_factor
        return "N/A"

    def b_factor(self):
        b_factors = []
        for model in self.structure:
            for chain in model:
                for residue in chain:
                    if 'CA' in residue:
                        b_factors.append(residue['CA'].get_bfactor())
        return sum(b_factors) / len(b_factors) if b_factors else "N/A"


    def sasa(self):
        # Ensure the correct file path is used
        structure = freesasa.Structure(self.file_path)
        result = freesasa.calc(structure)

        sasa_per_residue = defaultdict(float)
        for i in range(structure.nAtoms()):
            resname = structure.residueName(i)
            resnum = structure.residueNumber(i)
            chain = structure.chainLabel(i)
            sasa = result.atomArea(i)
            residue_key = f"{chain}{resnum} {resname}"

            # Accumulate SASA values for each residue
            sasa_per_residue[residue_key] += sasa

        # Convert the defaultdict to a list of dictionaries
        sasa_per_residue_list = [{'residue_key': key, 'sasa': value} for key, value in sasa_per_residue.items()]

        return sasa_per_residue_list



In [ ]:
import os

# Assuming PDBProcessor is already defined with all the methods like protein_name(), polymer_entity(), etc.

# Specify the folder containing your PDB files
folder_path = '/content/drive/MyDrive/PDBFILES/PDBs'  # Replace with your folder path

# Check if the folder path exists
if not os.path.exists(folder_path):
    print(f"The folder path {folder_path} does not exist. Please check the path and try again.")
else:
    # List all PDB files in the folder
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.pdb')]

    if not file_paths:
        print(f"No PDB files found in the folder {folder_path}.")
    else:
        # Iterate through each file in the folder and process it
        for file_path in file_paths:
            print(f"Processing file: {file_path}")
            processor = PDBProcessor(file_path)
            try:
                print(f"Protein Name: {processor.protein_name()}")
                print(f"Polymer Entity: {processor.polymer_entity()}")
                print(f"Sequence: {processor.sequence()}")
                print(f"C Alpha Coordinates: {processor.c_alpha_coords()}")
                print(f"Refinement Resolution: {processor.refinement_resolution()}")
                print(f"Experiment Type: {processor.experiment_type()}")
                print(f"Enzyme Classification: {processor.enzyme_classification()}")
                print(f"R Factor: {processor.r_factor()}")
                print(f"B Factor: {processor.b_factor()}")
                print(f"Symmetry Type: {processor.symmetry_type()}")
                print(f"SASA: {processor.sasa()}")
            except Exception as e:
                print(f"An error occurred while processing {file_path}: {e}")
            print()  # Add a blank line between each file's output for readability


Processing file: /content/drive/MyDrive/PDBFILES/PDBs/1H0J.pdb
Protein Name:  
Polymer Entity: Protein
Sequence: {'Protein': 'LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSSLLVKYVCCNTDRCN'}
C Alpha Coordinates: {'A': [array([47.873, 32.351, 21.457], dtype=float32), array([46.719, 29.304, 23.415], dtype=float32), array([43.076, 28.156, 23.346], dtype=float32), array([41.247, 25.443, 25.275], dtype=float32), array([39.753, 22.396, 23.56 ], dtype=float32), array([36.237, 20.99 , 24.122], dtype=float32), array([37.526, 19.745, 27.471], dtype=float32), array([39.108, 22.915, 29.02 ], dtype=float32), array([41.704, 20.71 , 30.731], dtype=float32), array([43.36 , 20.398, 27.315], dtype=float32), array([44.633, 23.165, 25.033], dtype=float32), array([46.778, 24.016, 22.034], dtype=float32), array([48.697, 26.881, 20.466], dtype=float32), array([46.816, 28.405, 17.546], dtype=float32), array([48.24 , 28.125, 14.017], dtype=float32), array([49.022, 31.068, 11.755], dtype=float32), array([45.816, 

In [ ]:
import os
import csv

def update_csv_with_pdb_data(pdb_folder, csv_file_path):
    method_names = [
        'protein_name', 'polymer_entity', 'sequence', 'c_alpha_coords',
        'refinement_resolution', 'experiment_type', 'enzyme_classification',
        'symmetry_type', 'r_factor', 'b_factor', 'sasa'  # Note lowercase 'sasa' as per your class method
    ]

    pdb_files = [f for f in os.listdir(pdb_folder) if f.endswith('.pdb')]

    if not pdb_files:
        print("No PDB files found in the directory.")
        return

    file_exists = os.path.isfile(csv_file_path)

    with open(csv_file_path, mode='a', newline='') as csvfile:
        fieldnames = ['file_name'] + method_names
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        for pdb_file in pdb_files:
            pdb_path = os.path.join(pdb_folder, pdb_file)
            processor = PDBProcessor(pdb_path)
            pdb_data = {'file_name': pdb_file}
            for method_name in method_names:
                method = getattr(processor, method_name)
                try:
                    result = method()
                    if method_name == 'sasa':  # Special handling for SASA data
                        result = ', '.join([f"{res['residue_key']}: {res['sasa']}" for res in result])
                    pdb_data[method_name] = result
                except Exception as e:
                    print(f"Error calling {method_name} on {pdb_file}: {e}")
                    pdb_data[method_name] = "Error"
            writer.writerow(pdb_data)

# Example usage
update_csv_with_pdb_data('/content/drive/MyDrive/PDBFILES/PDBs', '/content/drive/MyDrive/EmptyCSV.csv')


In [ ]:
import pandas as pd
dataframe = pd.read_csv('/content/drive/MyDrive/EmptyCSV.csv')

In [ ]:
dataframe.head()

,file_name,protein_name,polymer_entity,sequence,c_alpha_coords,refinement_resolution,experiment_type,enzyme_classification,symmetry_type,r_factor,b_factor,sasa
0,1H0J.pdb,,Protein,{'Protein': 'LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVP...,"{'A': [array([47.873, 32.351, 21.457], dtype=f...",1.90,x-ray diffraction,NaN,P 21 21 2 12,0.227,28.947833,"A 1 LEU: 65.16777016823494, A 2 LYS: 86...."
1,1H1L.pdb,,Protein,{'Protein': 'SQTIDKINSCYPLFEQDEYQELFRNKRQLEEAH...,"{'A': [array([33.993, 36.681, 19.646], dtype=f...",1.90,x-ray diffraction,NaN,C 1 2 1 8,NaN,21.959995,"A 1 THR: 102.1466993474299, A 2 ASN: 55...."
2,1H4G.pdb,H_PCA,Protein,{'Protein': 'IVTDNSIGNHDGYDYEFWKDSGGSGTMILNHGG...,"{'A': [array([12.179, 12.348, 29.948], dtype=f...",1.10,x-ray diffraction,NaN,P 21 21 21 8,NaN,11.565908,"A 2 ILE: 162.39634401943505, A 3 VAL: 14..."
3,5QTS.pdb,,Protein,{'Protein': 'ENARPKPKPGDGEFVEVISLPKNDLLQRLDALV...,"{'A': [array([15.312, -0.845, -2.272], dtype=f...",1.85,x-ray diffraction,NaN,P 1 4,0.218,35.291561,"A 14 LYS: 175.37714286045178, A 15 GLN: 19..."
4,5QTO.pdb,,Protein,{'Protein': 'ENARPKPKPGDGEFVEVISLPKNDLLQRLDALV...,"{'A': [array([15.406, -0.925, -2.285], dtype=f...",1.67,x-ray diffraction,NaN,P 1 4,0.231,38.800090,"A 14 LYS: 170.04531952886657, A 15 GLN: 20..."
